In [ ]:
%pip install --upgrade -r requirements.txt
%pip install rp --upgrade
# You may need to restart the runtime after installing these
# I'm not sure why this helps, but all sorts of weird random errors pop up in Colab if you don't

In [ ]:
import numpy as np
import rp
import torch
import torch.nn as nn
import source.stable_diffusion as sd
from easydict import EasyDict
from source.learnable_textures import LearnableImageFourier
from source.stable_diffusion_labels import NegativeLabel
from itertools import chain
import time
import torchvision.transforms.functional as TF
import math

In [ ]:
import os
# 确保你已经安装了 modelscope: pip install modelscope
if 's' not in dir():
    # --- 1. 配置模型保存的路径 ---
    # 使用相对路径（例如当前目录下的 weights 文件夹），这样换电脑也能直接用
    model_name = "./weights/sd-v1-4"
    # --- 2. 自动检查并下载逻辑 ---
    if not os.path.exists(model_name):
        print(f"模型路径 {model_name} 不存在，正在通过 ModelScope 自动下载...")
        try:
            from modelscope import snapshot_download
            snapshot_download('AI-ModelScope/stable-diffusion-v1-4', local_dir=model_name)
            print("模型下载完成！")
        except ImportError:
            raise ImportError("请先运行 pip install modelscope 以支持自动下载功能")
    else:
        print(f"检测到本地模型: {model_name}")
gpu=rp.select_torch_device()
s=sd.StableDiffusion(gpu,model_name)
device=s.device

In [ ]:
prompts_list = [
    "a snowy rocky mountain",          # Static View (原图)
     "a giant panda face"    # Motion Blurred View (晃动后)
]

# 动态模糊设置
BLUR_ANGLE = -45.0       # 晃动角度 (45度对角线)
BLUR_KERNEL_SIZE = 31   # 模糊核大小 (越大越模糊)
SIZE = 256              # 图片分辨率

negative_prompt = 'ugly, inconsistent colors'
print(f"\n=== Motion Blur Illusion Setup ===")
print(f"Angle: {BLUR_ANGLE}°, Kernel Size: {BLUR_KERNEL_SIZE}")
print(f"View 1 (Static): {prompts_list[0]}")
print(f"View 2 (Motion): {prompts_list[1]}")

labels = [NegativeLabel(p, negative_prompt) for p in prompts_list]
N = len(prompts_list)

In [ ]:
import torch.nn.functional as F

learnable_image_maker = lambda: LearnableImageFourier(height=SIZE, width=SIZE, hidden_dim=256, num_features=128).to(s.device)
main_image = learnable_image_maker()

# 创建运动模糊核
def get_motion_blur_kernel(kernel_size, angle_degrees):
    # 1. 生成水平线
    kernel = torch.zeros((kernel_size, kernel_size))
    center = kernel_size // 2
    kernel[center, :] = 1.0
    
    # 2. 旋转
    kernel = kernel.unsqueeze(0).unsqueeze(0) # [1,1,H,W]
    kernel = TF.rotate(kernel, angle_degrees, interpolation=TF.InterpolationMode.BILINEAR)
    kernel = kernel / kernel.sum() # 归一化
    
    kernel = kernel.repeat(3, 1, 1, 1)
    return kernel.to(device)

blur_kernel = get_motion_blur_kernel(BLUR_KERNEL_SIZE, BLUR_ANGLE)

# 索引 0: 返回原图
# 索引 1: 返回模糊后的图
learnable_images = [
    (lambda: main_image()), 
    (lambda: F.conv2d(main_image(), blur_kernel, padding=BLUR_KERNEL_SIZE//2, groups=3))
]

params = chain(main_image.parameters())
optim = torch.optim.SGD(params, lr=1e-4)

# 权重设置 (通常模糊后的 Loss 需要大一点)
weights = [1.0, 1.5] 
weights = rp.as_numpy_array(weights)
weights = weights / weights.sum() * len(weights) # Normalize

In [ ]:
ims = []

def get_display_image():
    current_views = [rp.as_numpy_image(img_func()) for img_func in learnable_images]
    
    final_image = rp.tiled_images(
        current_views, 
        length=2, 
        border_thickness=5, 
        border_color=(255,255,255)
    )
    return final_image

In [ ]:
NUM_ITER = 4000         
DISPLAY_INTERVAL = 200   

s.max_step = 980
s.min_step = 10 

display_eta = rp.eta(NUM_ITER, title='Status: ')

print(f'Starting training for Motion Blur Illusion...')
print(f'Left Image: Static ({prompts_list[0]})')
print(f'Right Image: Motion Blurred ({prompts_list[1]})')
print(f'Check output every {DISPLAY_INTERVAL} iterations (History kept).')

try:
    for iter_num in range(NUM_ITER):
        display_eta(iter_num) 

        preds = []

        for label, learnable_image_func, weight in rp.random_batch(list(zip(labels, learnable_images, weights)), batch_size=1):
            pred = s.train_step(
                label.embedding,
                learnable_image_func()[None], # 调用 lambda 获取当前视角的 tensor
                noise_coef=0.1 * weight,
                guidance_scale=60,
            )
            preds += list(pred)

        with torch.no_grad():
            if not iter_num % DISPLAY_INTERVAL:
                im = get_display_image()
                ims.append(im)
                print(f"\n--- Iteration {iter_num} ---")
                rp.display_image(im)

        optim.step()
        optim.zero_grad()

except KeyboardInterrupt:
    print()
    print('Interrupted early at iteration %i' % iter_num)
    im = get_display_image()
    ims.append(im)
    rp.display_image(im)

In [ ]:
print("=== Final Result ===")
final_static_tensor = main_image()
final_blurred_tensor = F.conv2d(final_static_tensor, blur_kernel, padding=BLUR_KERNEL_SIZE//2, groups=3)
img_static = rp.as_numpy_image(final_static_tensor)
img_blurred = rp.as_numpy_image(final_blurred_tensor)
print(f"\n[1] Static View (Should look like: {prompts_list[0]})")
rp.display_image(img_static)
print(f"\n[2] Motion Blurred View (Angle: {BLUR_ANGLE}°, Should look like: {prompts_list[1]})")
rp.display_image(img_blurred)
print("\n[3] Side-by-Side Comparison")
comparison = rp.tiled_images([img_static, img_blurred], length=2)
rp.display_image(comparison)